In [2]:
#importing
#!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import pandas as pd

In [27]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-27-0269635a80f9>:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [33]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("ga8").getOrCreate()

In [34]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator


In [35]:
test_scv_path = "gs://mukesh-iitm/mnist_train.csv"
data_df = spark.read.csv( test_scv_path, inferSchema=True ,header=True)

In [36]:
data_df.printSchema()

root
 |-- label: integer (nullable = true)
 |-- 1x1: integer (nullable = true)
 |-- 1x2: integer (nullable = true)
 |-- 1x3: integer (nullable = true)
 |-- 1x4: integer (nullable = true)
 |-- 1x5: integer (nullable = true)
 |-- 1x6: integer (nullable = true)
 |-- 1x7: integer (nullable = true)
 |-- 1x8: integer (nullable = true)
 |-- 1x9: integer (nullable = true)
 |-- 1x10: integer (nullable = true)
 |-- 1x11: integer (nullable = true)
 |-- 1x12: integer (nullable = true)
 |-- 1x13: integer (nullable = true)
 |-- 1x14: integer (nullable = true)
 |-- 1x15: integer (nullable = true)
 |-- 1x16: integer (nullable = true)
 |-- 1x17: integer (nullable = true)
 |-- 1x18: integer (nullable = true)
 |-- 1x19: integer (nullable = true)
 |-- 1x20: integer (nullable = true)
 |-- 1x21: integer (nullable = true)
 |-- 1x22: integer (nullable = true)
 |-- 1x23: integer (nullable = true)
 |-- 1x24: integer (nullable = true)
 |-- 1x25: integer (nullable = true)
 |-- 1x26: integer (nullable = true)
 |--

In [37]:
data_df.columns

['label',
 '1x1',
 '1x2',
 '1x3',
 '1x4',
 '1x5',
 '1x6',
 '1x7',
 '1x8',
 '1x9',
 '1x10',
 '1x11',
 '1x12',
 '1x13',
 '1x14',
 '1x15',
 '1x16',
 '1x17',
 '1x18',
 '1x19',
 '1x20',
 '1x21',
 '1x22',
 '1x23',
 '1x24',
 '1x25',
 '1x26',
 '1x27',
 '1x28',
 '2x1',
 '2x2',
 '2x3',
 '2x4',
 '2x5',
 '2x6',
 '2x7',
 '2x8',
 '2x9',
 '2x10',
 '2x11',
 '2x12',
 '2x13',
 '2x14',
 '2x15',
 '2x16',
 '2x17',
 '2x18',
 '2x19',
 '2x20',
 '2x21',
 '2x22',
 '2x23',
 '2x24',
 '2x25',
 '2x26',
 '2x27',
 '2x28',
 '3x1',
 '3x2',
 '3x3',
 '3x4',
 '3x5',
 '3x6',
 '3x7',
 '3x8',
 '3x9',
 '3x10',
 '3x11',
 '3x12',
 '3x13',
 '3x14',
 '3x15',
 '3x16',
 '3x17',
 '3x18',
 '3x19',
 '3x20',
 '3x21',
 '3x22',
 '3x23',
 '3x24',
 '3x25',
 '3x26',
 '3x27',
 '3x28',
 '4x1',
 '4x2',
 '4x3',
 '4x4',
 '4x5',
 '4x6',
 '4x7',
 '4x8',
 '4x9',
 '4x10',
 '4x11',
 '4x12',
 '4x13',
 '4x14',
 '4x15',
 '4x16',
 '4x17',
 '4x18',
 '4x19',
 '4x20',
 '4x21',
 '4x22',
 '4x23',
 '4x24',
 '4x25',
 '4x26',
 '4x27',
 '4x28',
 '5x1',
 '5x2',
 '

In [32]:
data_df.columns[1:]

['1x1',
 '1x2',
 '1x3',
 '1x4',
 '1x5',
 '1x6',
 '1x7',
 '1x8',
 '1x9',
 '1x10',
 '1x11',
 '1x12',
 '1x13',
 '1x14',
 '1x15',
 '1x16',
 '1x17',
 '1x18',
 '1x19',
 '1x20',
 '1x21',
 '1x22',
 '1x23',
 '1x24',
 '1x25',
 '1x26',
 '1x27',
 '1x28',
 '2x1',
 '2x2',
 '2x3',
 '2x4',
 '2x5',
 '2x6',
 '2x7',
 '2x8',
 '2x9',
 '2x10',
 '2x11',
 '2x12',
 '2x13',
 '2x14',
 '2x15',
 '2x16',
 '2x17',
 '2x18',
 '2x19',
 '2x20',
 '2x21',
 '2x22',
 '2x23',
 '2x24',
 '2x25',
 '2x26',
 '2x27',
 '2x28',
 '3x1',
 '3x2',
 '3x3',
 '3x4',
 '3x5',
 '3x6',
 '3x7',
 '3x8',
 '3x9',
 '3x10',
 '3x11',
 '3x12',
 '3x13',
 '3x14',
 '3x15',
 '3x16',
 '3x17',
 '3x18',
 '3x19',
 '3x20',
 '3x21',
 '3x22',
 '3x23',
 '3x24',
 '3x25',
 '3x26',
 '3x27',
 '3x28',
 '4x1',
 '4x2',
 '4x3',
 '4x4',
 '4x5',
 '4x6',
 '4x7',
 '4x8',
 '4x9',
 '4x10',
 '4x11',
 '4x12',
 '4x13',
 '4x14',
 '4x15',
 '4x16',
 '4x17',
 '4x18',
 '4x19',
 '4x20',
 '4x21',
 '4x22',
 '4x23',
 '4x24',
 '4x25',
 '4x26',
 '4x27',
 '4x28',
 '5x1',
 '5x2',
 '5x3',
 '5x

In [38]:
assembler = VectorAssembler(inputCols=data_df.columns[1:],
                            outputCol="features")
data_df = assembler.transform(data_df).select("label", "features")

In [39]:
data_df.select("label", "features").show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    5|(784,[152,153,154...|
|    0|(784,[127,128,129...|
|    4|(784,[160,161,162...|
|    1|(784,[158,159,160...|
|    9|(784,[208,209,210...|
|    2|(784,[155,156,157...|
|    1|(784,[124,125,126...|
|    3|(784,[151,152,153...|
|    1|(784,[152,153,154...|
|    4|(784,[134,135,161...|
|    3|(784,[123,124,125...|
|    5|(784,[216,217,218...|
|    3|(784,[143,144,145...|
|    6|(784,[72,73,74,99...|
|    1|(784,[151,152,153...|
|    7|(784,[211,212,213...|
|    2|(784,[151,152,153...|
|    8|(784,[159,160,161...|
|    6|(784,[100,101,102...|
|    9|(784,[209,210,211...|
+-----+--------------------+
only showing top 20 rows



In [41]:
distinct_labels = data_df.select("label").distinct().collect()
print(len(distinct_labels))


10


In [42]:
train_df, test_df= data_df.randomSplit([0.7, 0.3],seed=1234)

In [43]:
dtc = DecisionTreeClassifier(labelCol="label", featuresCol="features")

In [11]:
paramGrid = ParamGridBuilder() \
  .addGrid(dtc.maxDepth, [2, 3, 4, 5,6,8,10]) \
  .addGrid(dtc.maxBins, [10, 20, 30]) \
  .build()

In [44]:
evaluator=MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

In [45]:
crossval = CrossValidator(estimator=dtc,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)

In [ ]:
model = crossval.fit(train_df)

23/04/22 11:29:25 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_6524_1 !
23/04/22 11:29:25 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_6470_0 !
23/04/22 11:29:25 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_6567_0 !
23/04/22 11:29:25 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_6567_1 !
23/04/22 11:29:25 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_6524_0 !
23/04/22 11:29:25 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_6470_1 !
23/04/22 11:29:26 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1682157320725_0001_01_000004 on host: mlibcluster-m.us-central1-f.c.iitm-assigment-1.internal. Exit status: 137. Diagnostics: [2023-04-22 11:29:25.778]Container killed on reque

In [15]:
bestModel = model.bestModel

In [18]:
bestModel

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_9ef023cc333b, depth=10, numNodes=1605, numClasses=10, numFeatures=784

In [16]:
predictions = bestModel.transform(test_df)

In [17]:
predictions

DataFrame[label: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [19]:
accuracy = evaluator.evaluate(predictions)

# Print the results
print("Test Error = %g " % (1.0 - accuracy))

Test Error = 0.145351 


In [20]:
print("Best Model Parameters: maxDepth = %d, maxBins = %d" % (bestModel._java_obj.getMaxDepth(), bestModel._java_obj.getMaxBins()))

Best Model Parameters: maxDepth = 10, maxBins = 10


In [23]:
#bestModel.save("gs://mukesh-iitm/models/ga8")